In [0]:
#!import "../DataModel/DataStructure"

In [0]:
using System.IO;
using System.Text;
using Systemorph.Vertex.DataSetReader.Csv;
using Systemorph.Vertex.Session;
using Systemorph.Vertex.Import.Builders;
using Systemorph.Vertex.Import.Mappings;
using Systemorph.Vertex.DataSetReader;
using Systemorph.Vertex.Export.Excel.Builders;
using Systemorph.Vertex.Export;
using Systemorph.Vertex.Export.Builders;
using Systemorph.Vertex.FileStorage;
using Systemorph.Vertex.Export.Builders.Interfaces;

public record StreamWrapper(Stream Stream, bool WillBeReused);

public static string ProcessNotification(this object obj) => obj is ActivityMessageNotification amn ? amn.Message : ""; 

In [0]:
public record ImportExportActivity : KeyedRecord
{
    public string Username {get; init;}

    public DateTime StartDateTime {get; init;}

    public DateTime EndDateTime {get; init;}

    public ActivityLogStatus Status {get; init;}

    public string Category {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] ErrorMessages {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] WarningMessages {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] InfoMessages {get; init;}

    public Guid SourceId {get; init;} // Check if converting to Guid causes issues. If not, leave Guid.

    public ImportExportActivity(ActivityLog log, ISessionVariable session)
    {
        Id = Guid.NewGuid();
        Username = session.User.Name;
        StartDateTime = log.StartDateTime;
        EndDateTime = log.FinishDateTime;
        Status = log.Status;
        ErrorMessages = log.Errors.Select(x => x.ProcessNotification()).Distinct().ToArray();
        WarningMessages = log.Warnings.Select(x => x.ProcessNotification()).Distinct().ToArray();
        InfoMessages = log.Infos.Select(x => x.ProcessNotification()).Distinct().ToArray();
    }

    public ImportExportActivity(Guid id)
    {
        Id = id;
    }

}

In [0]:
public abstract record KeyedImportExport : KeyedRecord
{
    public DateTime CreationTime {get; init;}
    public byte[] SerializedContent {get; init;}
    public uint? Length {get; init;}
    public string Format {get; init;}
    protected IDataSetImportVariable DataSetReader {get; set;}
    protected ISessionVariable Session {get; set;}
}

In [0]:
public record ExportFile : KeyedImportExport
{
    protected DocumentBuilder Builder {get; set;}

    public string Name {get; init;}

    public string ContentType {get; init;}

    public ExportFile(DocumentBuilder builder, IDataSetImportVariable importVariable, ISessionVariable session)
    {
        Builder = builder;
        DataSetReader = importVariable;
        Session = session;
        Id = Guid.NewGuid();
        CreationTime = DateTime.UtcNow;
    }

    public ExportFile(Guid id)
    {
        Id = id;
    }

    public async Task<ExportFile> InitializeExportDataAsync()
    {
        byte[] content;
        var mapping = await Builder.GetMappingAsync();
        var storage = mapping.Storage as IFileReadStorage;
        var stream = await storage.ReadAsync(mapping.FileName, Session.CancellationToken);
        using(MemoryStream ms = new MemoryStream())
        {
            await stream.CopyToAsync(ms);
            content = ms.ToArray();
            stream.Close();
            await stream.DisposeAsync();
        }
        return this with {Name = Path.GetFileName(mapping.FileName), 
                        ContentType = Path.GetExtension(mapping.FileName),
                        SerializedContent = content, 
                        Length = content == null ? null : (uint)content.Length,
                        Format = mapping.Format};
    }
}

In [0]:
public abstract record KeyedImport : KeyedImportExport
{
    protected ImportOptions Options {get; set;}

    public async Task<KeyedImport> InitializeImportDataAsync()
    {
        var stream = await GenerateStreamWrapperAsync();
        var formatAndContent = await GetInformationFromStreamAsync(stream);
        return this with{CreationTime = DateTime.UtcNow, 
                            Format = formatAndContent.Format ?? Options.Format,
                            SerializedContent = formatAndContent.Content,
                            Length = SerializedContent != null ? (uint)SerializedContent.Length : null,
                        };
    }

    private async Task<StreamWrapper> GenerateStreamWrapperAsync()
    {
        StreamWrapper stream = Options switch
        {
            FileImportOptions fio => new StreamWrapper(await fio.Storage.ReadAsync(fio.FileName, Session.CancellationToken), true),
            StreamImportOptions streamImportOptions => new StreamWrapper(streamImportOptions.Stream, false),
            StringImportOptions stringImportOptions => new StreamWrapper(new MemoryStream(Encoding.ASCII.GetBytes(stringImportOptions.Content)), true),
            DataSetImportOptions dataSetImportOptions => new StreamWrapper(new MemoryStream(Encoding.ASCII.GetBytes(DataSetCsvSerializer.Serialize(dataSetImportOptions.DataSet))), true),
            _ => null
        };
        return stream;
    }

    private async Task<(string Format, byte[] Content)> GetInformationFromStreamAsync(StreamWrapper stream)
    {
        byte[] content;
        string format;
        using (MemoryStream ms = new MemoryStream())
        {
            await stream.Stream.CopyToAsync(ms);
            content = ms.ToArray();
            ms.Position = 0;
            var dsRes = await DataSetReader.ReadFromStream(ms).ExecuteAsync();
            format = dsRes.Format;
            if (stream.WillBeReused)
                stream.Stream.Position = 0;
            else
            {
                stream.Stream.Close();
                await stream.Stream.DisposeAsync();
            }
        }
        return (format, content);
    }
}

In [0]:
public record ImportFile : KeyedImport
{
    public string Name {get; init;}

    public string Directory {get; init;}
    
    public string ContentType {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] Partition {get; init;}

    public string Source {get; init;}

    public ImportFile(FileImportOptions options, IDataSetImportVariable importVariable, ISessionVariable session)
    {
        Options = options;
        DataSetReader = importVariable;
        Session = session;
        string fileName = options.FileName; 
        Id = Guid.NewGuid();
        Name = Path.GetFileName(fileName);
        Directory = Path.GetDirectoryName(fileName); 
        ContentType = Path.GetExtension(fileName);
        Source = options.Storage.GetType().Name; 
        Partition =  GetInvolvedPartitions(options);
        // Andrey Katz: Options.TargetDataSource.Partion.GetCurrent(?? What do we put here, different classes might posess various partitions, e.g. Yield Curve has none ??)
    }

    public ImportFile(Guid id)
    {
        Id = id;
        Options = null;
        DataSetReader = null;
    }
    

    private string[] GetInvolvedPartitions(ImportOptions options)
    {
        // TODO
        //Andrey Katz: Get all the relevant partitions here 
        return null;
    }
}

In [0]:
public record ImportString : KeyedImport
{
    public string Content {get; init;}

    public ImportString(StringImportOptions options, IDataSetImportVariable importVariable, ISessionVariable session)
    {
        Options = options;
        DataSetReader = importVariable;
        Session = session;
        Id = Guid.NewGuid();
        Content = options.Content;
    }

    public ImportString(Guid id)
    {
        Id = id;
        Options = null;
        DataSetReader = null;
    }
    
}

In [0]:
public record ImportDataSet : KeyedImport
{
    public ImportDataSet(DataSetImportOptions options, IDataSetImportVariable importVariable, ISessionVariable session)
    {
        Session = session;
        DataSetReader = importVariable;
        Options = options;
        Id = Guid.NewGuid();
    }

    public ImportDataSet(Guid id)
    {
        Id = id;
        Options = null;
        DataSetReader = null;
    }
    
}

In [0]:
public record ImportStream : KeyedImport
{
    public ImportStream(StreamImportOptions options, IDataSetImportVariable importVariable, ISessionVariable session)
    {
        Session = session;
        DataSetReader = importVariable;
        Options = options;
        Id = Guid.NewGuid();
    }

    public ImportStream(Guid id)
    {
        Id = id;
        Options = null;
        DataSetReader = null;
    }
    
}

In [0]:
public static async Task<ActivityLog> ExecuteWithStoreActivityAsync(this ImportOptionsBuilder builder, 
                                                                                ISessionVariable session, 
                                                                                IDataSource dataSource, 
                                                                                IDataSetImportVariable importVariable)
{
    var log = await builder.ExecuteAsync();
    var options = builder.GetImportOptions();
    var activity = new ImportExportActivity(log, session);
    bool importSucceeded = true;
    switch(options)
    {
        case FileImportOptions fio:
            var importFile = new ImportFile(Guid.NewGuid());
            try
            {
                importFile = await (new ImportFile(fio, importVariable, session)).InitializeImportDataAsync() as ImportFile;
            }
            catch (Exception)
            {
                importSucceeded = false;
            }
            activity = activity with {SourceId = importFile.Id, 
                                        Category = "Import from File"};
            if (importSucceeded) await dataSource.UpdateAsync<ImportFile>(importFile.RepeatOnce());
            break;
        case StringImportOptions sgio:
            var importString = new ImportString(Guid.NewGuid());
            try
            {  
                importString = await (new ImportString(sgio, importVariable, session)).InitializeImportDataAsync() as ImportString;
            }
            catch (Exception)
            {
                importSucceeded = false;
            }
            activity = activity with {SourceId = importString.Id, 
                                    Category = "Import from String"};
            if (importSucceeded) await dataSource.UpdateAsync<ImportString>(importString.RepeatOnce());
            break;
        case StreamImportOptions smio:
            var importStream = new ImportStream(Guid.NewGuid());
            try
            {
                importStream = await (new ImportStream(smio, importVariable, session)).InitializeImportDataAsync() as ImportStream;
            }
            catch (Exception)
            {
                importSucceeded = false;
            }
            activity = activity with {SourceId = importStream.Id, 
                                    Category = "Import from Stream"};
            if (importSucceeded) await dataSource.UpdateAsync<ImportStream>(importStream.RepeatOnce());
            break;
        case DataSetImportOptions dsio:
            var importDataSet = new ImportDataSet(Guid.NewGuid());
            try
            {
                importDataSet = await (new ImportDataSet(dsio, importVariable, session)).InitializeImportDataAsync() as ImportDataSet;
            }
            catch(Exception)
            {
                importSucceeded = false;
            }
            activity = activity with {SourceId = importDataSet.Id, 
                                    Category = "Import from Data Set"};
            if (importSucceeded) await dataSource.UpdateAsync<ImportDataSet>(importDataSet.RepeatOnce());
            break;
        default:
            throw new Exception("Import Options object is not an instance of an appropriate class.");
            break;
    }                              
    await dataSource.UpdateAsync<ImportExportActivity>(activity.RepeatOnce());
    await dataSource.CommitAsync(); 
    return log;
}

In [0]:
public static async Task<ExportResult> ExecuteWithStoreActivityAsync(this IDocumentBuilder builder, 
                                                                    ISessionVariable session, 
                                                                    IDataSource dataSource, 
                                                                    IDataSetImportVariable importVariable)
{
    var exportResult = await builder.ExecuteAsync();
    var exportFile = await (new ExportFile(builder as DocumentBuilder, importVariable, session)).InitializeExportDataAsync();
    var activity = new ImportExportActivity(exportResult.ActivityLog, session) with {Category = "Export to File", 
                                                                                    SourceId = exportFile.Id};
    await dataSource.UpdateAsync<ExportFile>(exportFile.RepeatOnce());
    await dataSource.UpdateAsync<ImportExportActivity>(activity.RepeatOnce());
    await dataSource.CommitAsync();
    return exportResult;
}